In [1]:
# import packages
import zipfile
import urllib.request

import numpy as np
import sklearn.metrics
import sklearn.ensemble

from datasets import load_dataset

import fasttext

import seaborn as sns
sns.set_style('whitegrid')
sns.set_context('talk')

In [2]:
# load IMDB dataset
df = load_dataset('imdb')

Found cached dataset imdb (/Users/Jlonging/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
# preprocess the data
with open('train.txt', 'w') as f:
    for text, label in zip(df['train']['text'], df['train']['label']):
        f.write(f'__label__{label} {text}\n')

with open('test.txt', 'w') as f:
    for text, label in zip(df['test']['text'], df['test']['label']):
        f.write(f'__label__{label} {text}\n')

In [4]:
# model training
model = fasttext.train_supervised('train.txt')

Read 5M words
Number of words:  281132
Number of labels: 2
Progress: 100.0% words/sec/thread: 5138857 lr:  0.000000 avg.loss:  0.406950 ETA:   0h 0m 0s


In [5]:
# Evaluate the model
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

print_results(*model.test('test.txt'))

N	25000
P@1	0.857
R@1	0.857


In [7]:
# Predict labels for test data
pred_labels = []
true_labels = df['test']['label']

for text in df['test']['text']:
    pred = model.predict(text)
    pred_label = int(pred[0][0].replace('__label__', ''))
    pred_labels.append(pred_label)

# Generate classification report
print(sklearn.metrics.classification_report(true_labels, pred_labels, target_names=['negative', 'positive']))

              precision    recall  f1-score   support

    negative       0.84      0.89      0.86     12500
    positive       0.88      0.83      0.85     12500

    accuracy                           0.86     25000
   macro avg       0.86      0.86      0.86     25000
weighted avg       0.86      0.86      0.86     25000

